In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
# Get the standard deviation of the prediction
y_pred.std()

5.28140357655334

In [11]:
year = 2022
month = 2

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


In [35]:
df_result = pd.concat([pd.Series(df["ride_id"]).reset_index(drop = True), pd.Series(y_pred).reset_index(drop = True)], axis=1).rename({0:" pred"}, axis=1)

In [40]:
import os
output_file = os.getcwd()

In [41]:
output_file = abs
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

TypeError: Unable to read from object of type: <class 'builtin_function_or_method'>